<a href="https://colab.research.google.com/github/zzeeddd95/capstone-project/blob/master/SVR%2Blinearregamazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas_datareader as web


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
#Retrieve stock data
amzn = web.DataReader('AMZN', data_source='yahoo', start='2010-05-27', end='2020-07-06')
print(amzn.head())

                  High         Low        Open       Close   Volume   Adj Close
Date                                                                           
2010-05-27  126.849998  120.599998  124.980003  126.699997  4747700  126.699997
2010-05-28  127.599998  124.010002  126.070000  125.459999  3954800  125.459999
2010-06-01  126.570000  123.019997  124.970001  123.239998  3659500  123.239998
2010-06-02  126.430000  121.650002  124.019997  126.309998  4764700  126.309998
2010-06-03  129.149994  124.849998  126.250000  128.759995  5276900  128.759995


In [3]:
#Filter out data to only obtain close price (independant variable)
amzn = amzn[['Close']]
amzn

,Close
Date,
2010-05-27,126.699997
2010-05-28,125.459999
2010-06-01,123.239998
2010-06-02,126.309998
2010-06-03,128.759995
...,...
2020-06-29,2680.379883
2020-06-30,2758.820068
2020-07-01,2878.699951


In [4]:
#Create a variable to allow for predicting 'n days' into the future
future_out =30

In [5]:
#Create the dependant variable which will be shifted 'n' units up
amzn['Prediction'] = amzn[['Close']].shift(-future_out)
amzn.tail()

,Close,Prediction
Date,,
2020-06-29,2680.379883,NaN
2020-06-30,2758.820068,NaN
2020-07-01,2878.699951,NaN
2020-07-02,2890.300049,NaN
2020-07-06,3057.040039,NaN


In [6]:
#create independant data set by converting the dataframe to a numpy array
amzndf = np.array(amzn.drop(['Prediction'],1))
#Renove the last 'n' rows
amzndfx = amzndf[:-future_out] #therefore for every column, all of the rows will displayed - future out
amzndfx

array([[ 126.69999695],
       [ 125.45999908],
       [ 123.23999786],
       ...,
       [2449.33007812],
       [2497.93994141],
       [2446.73999023]])

In [7]:
#creat dependant dataset (prediction), all values including any Nan's
amzndfy = np.array(amzn['Prediction'])
#Filter this data by obtaining all values except the last 'n' rows
amzndfy = amzndfy[:-future_out]
amzndfy

array([ 119.51000214,  123.65000153,  123.30000305, ..., 2878.69995117,
       2890.30004883, 3057.04003906])

In [8]:
#split data into 80% training and 20%testing
amzndfx_train , amzndfx_test, amzndfy_train, amzndfy_test = train_test_split(amzndfx, amzndfy, test_size = 0.2)

In [9]:
#Create and train the SVR model
SVRamzn = SVR(kernel='rbf', C=1e3,gamma=0.1)
SVRamzn.fit(amzndfx_train, amzndfy_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [10]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
SVMconfidence = SVRamzn.score(amzndfx_test, amzndfy_test)
print("SVM confidence", SVMconfidence)

SVM confidence 0.9511531997465685


In [11]:
#Create and train the linear regression model
linear = LinearRegression()
#train the model
linear.fit(amzndfx_train, amzndfy_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
linearconfidence = linear.score(amzndfx_test, amzndfy_test)
print("Linear regression confidence", linearconfidence)

Linear regression confidence 0.9787192199191079


In [13]:
#set apple_forecast = to the last 30 rows of original dataset from close price column
amzn_forecast = np.array(amzn.drop(['Prediction'],1))[-future_out:]
print(amzn_forecast)


[[2436.87988281]
 [2421.86010742]
 [2410.38989258]
 [2401.10009766]
 [2442.37011719]
 [2471.04003906]
 [2472.40991211]
 [2478.39990234]
 [2460.60009766]
 [2483.        ]
 [2524.06005859]
 [2600.86010742]
 [2647.44995117]
 [2557.95996094]
 [2545.02001953]
 [2572.67993164]
 [2615.27001953]
 [2640.97998047]
 [2653.97998047]
 [2675.01000977]
 [2713.82006836]
 [2764.40991211]
 [2734.39990234]
 [2754.58007812]
 [2692.87011719]
 [2680.37988281]
 [2758.82006836]
 [2878.69995117]
 [2890.30004883]
 [3057.04003906]]


In [14]:
#Print the SVR predicted values for the upcoming 'n' days
SVRprediction = SVRamzn.predict(amzn_forecast)
print(SVRprediction)

[1171.74670084 1316.02660636 2578.46476244 1937.61709426 1179.60852665
 1588.12223834 1948.32758809 1316.02618564 1171.73693242 1172.03740879
 1171.73386945 1171.73386945 1171.73386945 1171.73386945 1171.73386945
 1171.73386945 1171.73386945 1171.73386945 1171.73386945 1171.73386945
 1171.73386945 1171.73386945 1171.73386945 1171.73386945 1171.73386945
 1171.73386945 1171.73386945 1171.73386945 1171.73386945 1171.73386945]


In [15]:
#Print the linear regression predicted values for the upcoming 'n' days
linearprediction = linear.predict(amzn_forecast)
print(linearprediction)

[2519.09696387 2503.60328516 2491.77116258 2482.18825644 2524.76042588
 2554.33493992 2555.74803515 2561.9270213  2543.56559793 2566.67226104
 2609.02784487 2688.25108559 2736.31093012 2643.9973555  2630.64913361
 2659.18177009 2703.11565864 2729.63681917 2743.04699463 2764.74056252
 2804.77515444 2856.9612068  2826.00432248 2846.82122231 2783.16419129
 2770.27986557 2851.19499254 2974.85732034 2986.82342379 3158.82438715]


In [17]:
#Retrieve stock data
amznstock = web.DataReader('AMZN', data_source='yahoo', start='2020-05-28', end='2020-07-05')
amznstock = amznstock[['Close']]
amznstock.head()

,Close
Date,
2020-05-28,2401.100098
2020-05-29,2442.370117
2020-06-01,2471.040039
2020-06-02,2472.409912
2020-06-03,2478.399902
